In [1]:
from collections import defaultdict
import random
import numpy as np
np.set_printoptions(precision=2, suppress=True)

import time
import copy 
import multiprocess as mp

import gym
from env import FrozenLakeCustom, FrozenLakeSimulator

from mcts_haver import run_mcts_trial
from value_iteration import value_iteration

from config import parse_args
from utils import MultiProcess

import logging
logger = logging.getLogger()
logger.setLevel(logging.FATAL)

In [2]:
np.random.seed(0)
random.seed(0)

# params
args = parse_args()

#
env_id = "FrozenLake-v1"
env = FrozenLakeCustom(
    map_name=args["map_name"], is_slippery=args["is_slippery"],
    render_mode=args["render_mode"])

simulator = FrozenLakeSimulator(env.P)

V_vit, Q_vit = value_iteration(
    simulator, args["gamma"], args["vit_thres"])
# global Q_vit_g = Q_vit
        
for state in range(simulator.num_states):
    logging.warning(f"\n-> state = {state}")
    logging.warning(f"V[state] = {V_vit[state]:0.4f}")
    for action in range(simulator.num_actions):
        logging.warning(f"Q[state][action] = {Q_vit[state][action]:0.4f}")
    logging.warning(f"best_action={np.argmax(Q_vit[state])}")
    
manager = mp.Manager()
ep_reward_list = manager.list()
Q_mcts_list = manager.list()

def run_trial(i_trial, Q_vit, args):

    random.seed(10000+i_trial)
    np.random.seed(10000+i_trial)

    env = FrozenLakeCustom(
        map_name=args["map_name"], is_slippery=args["is_slippery"],
        render_mode=args["render_mode"])

    simulator = FrozenLakeSimulator(env.P)

    Q_mcts, ep_reward = run_mcts_trial(env, simulator, Q_vit, i_trial, args)

    ep_reward_list.append(ep_reward)
    Q_mcts_list.append(Q_mcts)
    return ep_reward

In [3]:
args["update_method"] = "haver"
args["rollout_method"] = ""

print(f"num_trials = {args['num_trials']}")
# print(f"mcts_num_trajectories = {args['mcts_num_trajectories']}")


hparam_ucb_scale_list = np.arange(10, 100, 10)
hparam_ucb_scale_list = [1, 2, 4, 8, 16, 32, 64, 128]
# hparam_ucb_scale_list = [2**i for i in range(1, 9)]
args["hparam_ucb_scale"] = 64

hparam_haver_std_list = np.arange(10, 100, 10)
hparam_haver_std_list = [1, 2, 4, 8, 16, 32, 64, 128]
# hparam_haver_std_list = [2**i for i in range(1, 9)]


# num_trajectories_list = [200, 500, 1000, 1500, 2000, 2500, 3000]
num_trajectories_list = [200]
# num_trajectories_list = [2]
best_param_list = []
max_reward_mean_list = []
res_text1 = ""
res_text2 = ""
for num_trajectories in num_trajectories_list:
    print(f"\n-> num_trajectories = {num_trajectories}")
    args["mcts_num_trajectories"] = num_trajectories
    
    # best_param = None
    # max_reward_mean = -np.inf
    start_time = time.time()
    res_text1 += f"{num_trajectories} "
    res_text2 += f"{num_trajectories} "
    for hparam_ucb_scale in hparam_ucb_scale_list: 
        
        args["hparam_ucb_scale"] = hparam_ucb_scale
        print(f"\n-> hparam_ucb_scale = {hparam_ucb_scale}")
        
        max_reward_mean = -np.inf
        best_param = None
        max_reward_error = None
        
        for hparam_haver_std in hparam_haver_std_list:
            # start_time = time.time()

            args["hparam_haver_var"] = hparam_haver_std**2
            # print(f"hparam_haver_var = {args['hparam_haver_var']}")
            # print(f"hparam_ucb_scale = {args['hparam_ucb_scale']}")

            pool = mp.Pool()
            pool.starmap(run_trial, [(i, Q_vit, args) for i in range(args["num_trials"])])

            reward_mean = np.mean(ep_reward_list)
            reward_std = np.std(ep_reward_list, ddof=1) if len(ep_reward_list) > 1 else 0
            reward_error = reward_std/np.sqrt(args["num_trials"])
            # if hparam_haver_std <= 8:
            #     res_text1 += f"& {reward_mean:0.2f} (\u00B1{reward_error:0.2f}) "
            # else:
            #     res_text2 += f"& {reward_mean:0.2f} (\u00B1{reward_error:0.2f}) "
            print(f"reward = {reward_mean:0.2f} \u00B1 {reward_error:0.2f}")

            if reward_mean > max_reward_mean:
                max_reward_mean = reward_mean 
                max_reward_error = reward_error
                best_param = hparam_haver_std
                
            ep_reward_list[:] = []
            Q_mcts_list[:] = []

            end_time = time.time()
            # print(f"it takes {end_time-start_time:0.4f}")
        
        if hparam_ucb_scale <= 8:
            res_text1 += f"& {max_reward_mean:0.2f} (\u00B1{max_reward_error:0.2f}) "
        else:
            res_text2 += f"& {max_reward_mean:0.2f} (\u00B1{max_reward_error:0.2f}) "
            
        print(f"max_reward = {max_reward_mean:0.2f} \u00B1 {max_reward_error:0.2f}")
        print(f"best_param = {best_param}")
            
    res_text1 += "\\\\ \n \hline \n"
    res_text2 += "\\\\ \n \hline \n"

    # print(f"max_reward_mean = {max_reward_mean:0.2f}")
    print(f"it takes {end_time-start_time:0.4f}")

    max_reward_mean_list.append(max_reward_mean)
    best_param_list.append(best_param)

num_trials = 500

-> num_trajectories = 200

-> hparam_ucb_scale = 1
reward = -100.91 ± 0.19
reward = -100.90 ± 0.19
reward = -100.53 ± 0.34
reward = -100.56 ± 0.34
reward = -100.57 ± 0.34
reward = -100.36 ± 0.39
reward = -100.75 ± 0.28
reward = -100.75 ± 0.28
max_reward = -100.36 ± 0.39
best_param = 32

-> hparam_ucb_scale = 2
reward = -99.21 ± 0.60
reward = -99.39 ± 0.58
reward = -100.17 ± 0.43
reward = -100.16 ± 0.43
reward = -100.37 ± 0.39
reward = -99.78 ± 0.51
reward = -100.16 ± 0.43
reward = -100.36 ± 0.39
max_reward = -99.21 ± 0.60
best_param = 1

-> hparam_ucb_scale = 4
reward = -88.82 ± 1.45
reward = -88.66 ± 1.45
reward = -89.76 ± 1.39
reward = -92.27 ± 1.25
reward = -94.59 ± 1.09
reward = -93.40 ± 1.18
reward = -93.62 ± 1.16
reward = -93.43 ± 1.17
max_reward = -88.66 ± 1.45
best_param = 2

-> hparam_ucb_scale = 8
reward = -67.10 ± 2.07
reward = -68.30 ± 2.05
reward = -65.04 ± 2.10
reward = -72.36 ± 1.98
reward = -76.37 ± 1.89
reward = -73.70 ± 1.95
reward = -78.77 ± 1.82
re

In [4]:
print(res_text1)
print(res_text2)

200 & -100.36 (±0.39) & -99.21 (±0.60) & -88.66 (±1.45) & -65.04 (±2.10) \\ 
 \hline 

200 & -41.75 (±2.11) & -28.84 (±1.89) & -26.28 (±1.82) & -23.19 (±1.72) \\ 
 \hline 



In [5]:
args["update_method"] = "haver"
args["rollout_method"] = ""

print(f"num_trials = {args['num_trials']}")
# print(f"mcts_num_trajectories = {args['mcts_num_trajectories']}")


hparam_ucb_scale_list = np.arange(10, 100, 10)
hparam_ucb_scale_list = [1, 2, 4, 8, 16, 32, 64, 128]
# hparam_ucb_scale_list = [2**i for i in range(1, 9)]
args["hparam_ucb_scale"] = 64

hparam_haver_std_list = np.arange(10, 100, 10)
hparam_haver_std_list = [1, 2, 4, 8, 16, 32, 64, 128]
# hparam_haver_std_list = [2**i for i in range(1, 9)]


# num_trajectories_list = [200, 500, 1000, 1500, 2000, 2500, 3000]
num_trajectories_list = [500]
# num_trajectories_list = [2]
best_param_list = []
max_reward_mean_list = []
res_text1 = ""
res_text2 = ""
for num_trajectories in num_trajectories_list:
    print(f"\n-> num_trajectories = {num_trajectories}")
    args["mcts_num_trajectories"] = num_trajectories
    
    # best_param = None
    # max_reward_mean = -np.inf
    start_time = time.time()
    res_text1 += f"{num_trajectories} "
    res_text2 += f"{num_trajectories} "
    for hparam_ucb_scale in hparam_ucb_scale_list: 
        
        args["hparam_ucb_scale"] = hparam_ucb_scale
        print(f"\n-> hparam_ucb_scale = {hparam_ucb_scale}")
        
        max_reward_mean = -np.inf
        best_param = None
        max_reward_error = None
        
        for hparam_haver_std in hparam_haver_std_list:
            # start_time = time.time()

            args["hparam_haver_var"] = hparam_haver_std**2
            # print(f"hparam_haver_var = {args['hparam_haver_var']}")
            # print(f"hparam_ucb_scale = {args['hparam_ucb_scale']}")

            pool = mp.Pool()
            pool.starmap(run_trial, [(i, Q_vit, args) for i in range(args["num_trials"])])

            reward_mean = np.mean(ep_reward_list)
            reward_std = np.std(ep_reward_list, ddof=1) if len(ep_reward_list) > 1 else 0
            reward_error = reward_std/np.sqrt(args["num_trials"])
            # if hparam_haver_std <= 8:
            #     res_text1 += f"& {reward_mean:0.2f} (\u00B1{reward_error:0.2f}) "
            # else:
            #     res_text2 += f"& {reward_mean:0.2f} (\u00B1{reward_error:0.2f}) "
            print(f"reward = {reward_mean:0.2f} \u00B1 {reward_error:0.2f}")

            if reward_mean > max_reward_mean:
                max_reward_mean = reward_mean 
                max_reward_error = reward_error
                best_param = hparam_haver_std
                
            ep_reward_list[:] = []
            Q_mcts_list[:] = []

            end_time = time.time()
            # print(f"it takes {end_time-start_time:0.4f}")
        
        if hparam_ucb_scale <= 8:
            res_text1 += f"& {max_reward_mean:0.2f} (\u00B1{max_reward_error:0.2f}) "
        else:
            res_text2 += f"& {max_reward_mean:0.2f} (\u00B1{max_reward_error:0.2f}) "
            
        print(f"max_reward = {max_reward_mean:0.2f} \u00B1 {max_reward_error:0.2f}")
        print(f"best_param = {best_param}")
            
    res_text1 += "\\\\ \n \hline \n"
    res_text2 += "\\\\ \n \hline \n"

    # print(f"max_reward_mean = {max_reward_mean:0.2f}")
    print(f"it takes {end_time-start_time:0.4f}")

    max_reward_mean_list.append(max_reward_mean)
    best_param_list.append(best_param)

num_trials = 500

-> num_trajectories = 500

-> hparam_ucb_scale = 1
reward = -100.70 ± 0.27
reward = -100.89 ± 0.19
reward = -100.88 ± 0.19
reward = -100.91 ± 0.19
reward = -100.71 ± 0.27
reward = -100.91 ± 0.19
reward = -100.90 ± 0.19
reward = -100.90 ± 0.19
max_reward = -100.70 ± 0.27
best_param = 1

-> hparam_ucb_scale = 2
reward = -97.82 ± 0.79
reward = -97.60 ± 0.81
reward = -100.30 ± 0.39
reward = -98.98 ± 0.63
reward = -99.60 ± 0.54
reward = -99.61 ± 0.54
reward = -99.77 ± 0.51
reward = -99.97 ± 0.47
max_reward = -97.60 ± 0.81
best_param = 2

-> hparam_ucb_scale = 4
reward = -83.70 ± 1.67
reward = -81.61 ± 1.74
reward = -86.45 ± 1.55
reward = -86.08 ± 1.57
reward = -88.62 ± 1.45
reward = -89.38 ± 1.42
reward = -90.36 ± 1.36
reward = -90.56 ± 1.35
max_reward = -81.61 ± 1.74
best_param = 2

-> hparam_ucb_scale = 8
reward = -44.66 ± 2.13
reward = -44.81 ± 2.13
reward = -46.99 ± 2.15
reward = -49.60 ± 2.16
reward = -50.65 ± 2.16
reward = -53.21 ± 2.16
reward = -54.70 ± 2.16
reward 

In [6]:
print(res_text1)
print(res_text2)

500 & -100.70 (±0.27) & -97.60 (±0.81) & -81.61 (±1.74) & -44.66 (±2.13) \\ 
 \hline 

500 & -28.26 (±1.87) & -17.65 (±1.51) & -18.75 (±1.56) & -15.65 (±1.41) \\ 
 \hline 

